# Demonstration: calc_stacking_fault

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Chandler A. Becker**, [chandler.becker@nist.gov](mailto:chandler.becker@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Zachary T. Trautt**, [zachary.trautt@nist.gov](mailto:zachary.trautt@nist.gov?Subject=ipr-demo), *Materials Measurement Science Division, NIST*.

Version: 2016-07-08

[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm)

- - -

## Software

This notebook was tested with:

- LAMMPS (Version 2016-03-11)

- Python (Version 2.7.11)

- Jupyter (Version 1.0.0)

__NOTE__: The potential embedded in this Notebook is a MEAM potential.  Running this script with the embedded potential requires that the LAMMPS executable being used has been built with the optional meam package. 

- - -

## Introduction

This Notebook outlines the code performed by the stacking_fault.py calculation and offers a fully self-contained demonstration of the calculation in action.

The calculation script determines the stacking fault energy associated with displacing two sections of a lattice from one another. The system is set up such that it has a free surface boundary condition along one direction and is periodic along the other two directions. The atoms are only allowed to relax in the direction normal to the plane of separation. By calculating the cohesive energy of the system before and after the calculation, the stacking fault energy can be calculated. 

__NOTE__: With this method there is no guarantee that the resulting parameters are for a stable structure. Allowing internal relaxations may result in different values for some structures. Additionally, some transformation paths may be restricted from ocurring, which allows for unstable structures to be refined.  

- - -

## 1. Initial Setup

1. __Necessary Python Libraries__: This is a list of the Python libraries used to run this code. 

2. __Set Working Directory__: Change the directory for performing the calculation.

3. __Input Parameter File__: An embedded copy of the input file read by the script is described.


### 1.1 Necessary Python Libraries

This is a list of the Python libraries used to run this code. 

- [numpy](http://www.numpy.org/) (Version 1.10.04)

- [atomman](https://github.com/usnistgov/atomman) (Version 0.6)

- [DataModelDict](https://github.com/usnistgov/DataModelDict) (Version 0.8)

In [1]:
#Standard Python Libraries
import os
import sys
import random
import uuid
from copy import deepcopy

#Additional Python Libraries
import iprPy
from DataModelDict import DataModelDict as DM
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc
import matplotlib.pyplot as plt
import numpy as np
from atomman import Atoms, Box, System

## 1.2 Set Working Directory

This allows for the working directory of the calculation to be changed.

In [2]:
#Specify working directory
working_dir = 'calc_stacking_fault_static'

#Check if a relative working_dir matches the current directory's name.
#This ensures that recursive directories are not created if this cell is called multiple times.
cwd = os.path.basename(os.getcwd())
if cwd != working_dir:

    #Move to working_dir and create if needed
    working_dir = os.path.abspath(working_dir)
    try:
        os.chdir(working_dir)
    except:
        os.makedirs(working_dir)
        os.chdir(working_dir)

#os.chdir('C:\Users\\nsl2\Documents\iprPy-develop\iprPy-develop\iprPy\calculations\stacking_fault_static\calc_files') #delete later

## 1.3 Input Parameter File

This code creates a copy of the input file as read in by the calculation script.  The input command keywords for this calculation are:

 - __lammps_command__: the path to the executable for running LAMMPS on your system.
 - __mpi_command__: the command associated with calling LAMMPS to run in parallel on your system (optional).
 - __potential_file__: the path to the LAMMPS-potential data model used by atomman to generate the proper LAMMPS commands for an interatomic potential.
 - __potential_dir__: the path to the directory containing any potential artifacts (eg. eam setfl files) that are used. Default is '' (working directory for the simulation.)
 - __load__: the style and path to the initial configuration file being read in. The style can be any file type supported by atomman.System.load()
 - __load_options__: a list of key-value pairs for the optional arguments in atomman.System.load()
 - __symbols__: a space-delimited list of elemental symbols corresponding to the atom types and potential.  Is optional if the elemental information is contained in the file indicated in load.
 - __stacking_fault_model__: the path to the directory containing .json files specifying the crystallographic orientations of the x-, y-, and z- axes.
 - __shift_amount__: (x y z) parameters that specify the extent of atomic displacements. Must match up with the axis specified in __cutting_axis__ (e.g. if 'z' is specified in __cutting_axis__, the z-component of __shift_amount__ must be 0).
 - __cutting_axis__: the normal vector direction (x, y, or z) to the desired cutting plane (e.g. 'z' would displace atoms along a plane parallel to the xy plane).
 - __cutting_plane_location__: a float between 0 and 1 that specifies the location of the cutting plane along the cutting axis as a percentage of the axis dimension.
 - __size_mults__: optional parameters for supersizing the rotated system by. The input format is either 3 multiplicative numbers (x y z) specifying how much the system is to be extended along each axis (in a direction away from the origin) or 6 multiplicative numbers (x1 x2 y1 y2 z1 z2) that specify how far the system is to be extended in the negative and positive directions along each axis.
 - __box_parameters__: optional new box parameters (either a b c, or a b c alpha beta gamma) to scale the loaded system to.
 - __x-axis, y-axis, z-axis__: optional transformation axes for rotating the system. 
 - __atom_shift__: optional positional shift that is applied to all atoms.  Values are in fractional box vectors of the rotated system before size_mults are applied.
 - __length_unit__: defines the unit of length for results, and input parameters if not specified. Default is 'angstrom'.
 - __energy_unit__: defines the unit of energy for results, and input parameters if not specified. Default is 'eV'.


__NOTE__: The potential_file '2012--Jelinek-B--Al-Si-Mg-Cu-Fe.json' and its associated artifacts are embedded in this Notebook in Sections 2.1 and 2.2, respectively.  The load system_model file 'A1--Cu--fcc.json' is embedded in this Notebook in Section 2.3.  These embedded files are automatically generated when this Notebook is executed allowing for 

In [3]:
with open('calc_stacking_fault_static.in', 'w') as input_file:
          input_file.write("""
#Run script for calc_stacking_fault_static.py

#Command lines for LAMMPS (and MPI).
#For lammps_command, exclude passing in a script, i.e. no "-in term" or "< term"
lammps_command              lmp_serial
mpi_command                 

#Paths to the potential data model file, and directory containing potential parameters
potential_file              2012--Jelinek-B--Al-Si-Mg-Cu-Fe.json
potential_dir               

#Initial system configuration to load
load                        system_model A1--Cu--fcc.json
load_options                
symbols                     AlS

#Defect parameters
stacking_fault_model        111_stacking_fault.json
shift_amount                0 0.7 0
cutting_axis                z
cutting_plane_location      0.5

#System manipulations
size_mults                  0 2 0 2 0 6
box_parameters              4.0502 4.0502 4.0502

#Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

#Run parameters
energy_tolerance            
force_tolerance             
maximum_iterations          
maximum_evaluations
""")

## 2. Additional Data Files

To make this notebook self-contained, we have embedded a few files.

1. __MEAM Potential.__ Generates library and parameter files for one MEAM potential.

2. __Potential Data Model.__ Collects all input parameters assoiciated with running the MEAM potential in LAMMPS.

3. __Prototype Data Model.__ Generates an instance of the crystal prototype data model used to generate various structures.

### 2.1 MEAM Potential

This generates the library and parameter files associated with the following interatomic potential taken from the [NIST Interatomic Potential Repository](http://www.ctcms.nist.gov/potentials/):

__2012--Jelinek-B--Al-Si-Mg-Cu-Fe__

__Aluminum, Silicon, Magnesium, Copper, and Iron (Al, Si, Mg, Cu, and Fe) Alloys__

*B. Jelinek, S. Groh, M. Horstemeyer, J. Houze, S.G. Kim, G.J. Wagner, A. Moitra, and M.I. Baskes, "Modified embedded atom method potential for Al, Si, Mg, Cu, and Fe alloys," Phys. Rev. B 85, 245102 (2012).*

__Notes__: This file was provided by Bohumir Jelinek (Mississippi State University) and posted on 3 July 2012. He noted, "This is a MEAM potential for Al, Si, Mg, Cu, Fe alloys. It works with LAMMPS, version 19 Jul 2011 or later, when compiled with MEAM support."

__Notice__: Users should consider all available potentials and select one which is appropriate for their use case. Use of this potential within this demonstration should not be construed as an endorsement or a recommendation. 

In [4]:
#Create MEAM library and parameter files for the 2012--Jelinek-B--Al-Si-Mg-Cu-Fe potential
   
#Create MEAM library file "Jelinek_2012_meamf"
with open('Jelinek_2012_meamf', 'w') as f:
    f.write("""# MEAM Al, Si, Mg, Cu, Fe alloy potential, Phys. Rev. B 85, 245102 (2012)
# http://dx.doi.org/10.1103/PhysRevB.85.245102
# to be used with "meam.alsimgcufe" file providing combinational parameters
#  elt        lat     z       ielement     atwt
#  alpha      b0      b1      b2           b3    alat    esub    asub
#  t0         t1              t2           t3            rozero  ibar

'AlS'        'fcc'   12.     13           26.9815
4.64        2.04    3.00    6.0          1.50  4.05    3.353    1.07
1.0         +4.50           -2.30        8.01          1.0    -5
'SiS'        'dia'   4.      14           28.086
4.87        4.4     5.5     5.5          5.5   5.431   4.63    1.
1.0         2.05            4.47         -1.80         2.2    -5
'MgS'       'hcp'   12.      12           24.305
5.52        4.0    3.0     0.2          1.2  3.194 1.51     0.80
1.0         10.04           9.49         -4.3          0.63   -5
'CuS'        'fcc'   12.     29           63.54
5.11        3.634   2.20    6            2.20  3.62    3.54    1.07
1.0         4.91            2.49         2.95          1.10   -5
'FeS'        'bcc'   8       26           55.847
5.0270      3.500   2       1.00         1     2.851   4.28    0.5550
1          -1.6             12.5          -1.40        1.0    -5""")

#Create MEAM parameter file "Jelinek_2012_meam.alsimgcufe"
with open('Jelinek_2012_meam.alsimgcufe', 'w') as f:
    f.write("""# MEAM Al, Si, Mg, Cu, Fe alloy potential, Phys. Rev. B 85, 245102 (2012)
# http://dx.doi.org/10.1103/PhysRevB.85.245102
# to be used with "meamf" file providing single element parameters

  Cmin(1,1,1) = 0.8
  repuls(1,1) = 0.1
  Cmin(3,3,3) = 0.8
  Cmin(4,4,4) = 0.8
  Cmin(5,5,5) = 0.68
  repuls(5,5) = 0.3
  Cmax(5,5,5) = 1.9

  nn2(1,1)=1
  nn2(1,2)=1
  nn2(1,3)=1
  nn2(1,4)=1
  nn2(1,5)=1
  nn2(2,2)=1
  nn2(2,3)=1
  nn2(2,4)=1
  nn2(2,5)=1
  nn2(3,3)=1
  nn2(3,4)=1
  nn2(3,5)=1
  nn2(4,4)=1
  nn2(4,5)=1
  nn2(5,5)=1

  lattce(1,2)='b1'
  delta(1,2)=+0.28
  alpha(1,2)=4.56
  re(1,2)=2.62
  Cmin(1,1,2) = 0.50
  Cmin(2,2,1) = 2.00
  Cmin(1,2,1) = 2.00
  Cmin(1,2,2) = 2.00

  lattce(1,3)='b1'
  delta(1,3)=+0.23
  alpha(1,3)=4.52
  re(1,3)=2.87
  Cmin(1,1,3) = 2.00
  Cmin(3,3,1) = 0.00
  Cmin(1,3,1) = 2.00
  Cmin(1,3,3) = 0.00

  lattce(1,4)='b1'
  delta(1,4)=+0.19
  alpha(1,4)=4.65
  re(1,4)=2.53
  Cmin(1,1,4) = 0.00
  Cmin(4,4,1) = 2.00
  Cmin(1,4,1) = 2.00
  Cmin(1,4,4) = 2.00

  lattce(1,5)='b1'
  delta(1,5)=+0.26
  alpha(1,5)=4.64
  re(1,5)=2.45
  Cmin(1,1,5) = 0.90
  Cmin(5,5,1) = 0.10
  Cmin(1,5,1) = 2.00
  Cmin(1,5,5) = 2.00

  lattce(2,3)='b1'
  delta(2,3)=+0.2
  alpha(2,3)=4.73
  re(2,3)=2.75
  Cmin(2,2,3) = 1.00
  Cmin(3,3,2) = 1.00
  Cmin(2,3,2) = 2.00
  Cmin(2,3,3) = 2.00

  lattce(2,4)='b1'
  delta(2,4)=+0.14
  alpha(2,4)=4.74
  re(2,4)=2.46
  Cmin(2,2,4) = 0.00
  Cmin(4,4,2) = 0.00
  Cmin(2,4,2) = 2.00
  Cmin(2,4,4) = 2.00

  lattce(2,5)='b1'
  delta(2,5)=-0.07
  alpha(2,5)=5.17
  re(2,5)=2.39
  Cmin(2,2,5) = 1.00
  Cmin(5,5,2) = 1.00
  Cmin(2,5,2) = 2.00
  Cmin(2,5,5) = 0.00
  attrac(2,5) = 0.1
  repuls(2,5) = 0.1

  lattce(3,4)='b1'
  delta(3,4)=+0.23
  alpha(3,4)=4.70
  re(3,4)=2.63
  Cmin(3,3,4) = 2.00
  Cmin(4,4,3) = 0.00
  Cmin(3,4,3) = 2.00
  Cmin(3,4,4) = 2.00

  lattce(3,5)='b1'
  delta(3,5)=+0.6
  alpha(3,5)=4.96
  re(3,5)=2.61
  Cmin(3,3,5) = 0.65
  Cmin(5,5,3) = 0.00
  Cmin(3,5,3) = 2.00
  Cmin(3,5,5) = 2.00

  lattce(4,5)='b1'
  delta(4,5)=+0.63
  alpha(4,5)=5.21
  re(4,5)=2.42
  Cmin(5,5,4)=0.00

  attrac(5,2) = 0.1
  repuls(5,2) = 0.1

  rc = 5.0
  ialloy=1
  augt1=0
  delr=0.25658351
  emb_lin_neg=1
  bkgd_dyn=1""")

### 2.2 Potential Data Model

This generates the interatomic potential data model file associated with the 2012--Jelinek-B--Al-Si-Mg-Cu-Fe potential.

In [5]:
#Creates a interatomic potential data model for using the MEAM potential in LAMMPS
with open('2012--Jelinek-B--Al-Si-Mg-Cu-Fe.json', 'w') as f:
    f.write("""{
    "LAMMPS-potential": {
        "potential": {
            "key": "1515dd80-1984-49de-bc92-d5724059ff56", 
            "id": "2012--Jelinek-B--Al-Si-Mg-Cu-Fe"
        }, 
        "units": "metal", 
        "atom_style": "atomic", 
        "atom": [
            {
                "symbol": "AlS",
                "element": "Al", 
                "mass": 26.9815                
            }, 
            {
                "symbol": "SiS",
                "element": "Si", 
                "mass": 28.086                
            }, 
            {
                "symbol": "MgS", 
                "element": "Mg",
                "mass": 24.305                
            }, 
            {
                "symbol": "CuS", 
                "element": "Cu", 
                "mass": 63.54
            }, 
            {
                "symbol": "FeS", 
                "element": "Fe", 
                "mass": 55.847
            }
        ], 
        "pair_style": {
            "type": "meam"
        }, 
        "pair_coeff": {
            "term": [
                {
                    "file": "Jelinek_2012_meamf"
                }, 
                {
                    "symbolsList": "True"
                }, 
                {
                    "file": "Jelinek_2012_meam.alsimgcufe"
                }, 
                {
                    "symbols": "True"
                }
            ]
        }
    }
}""")

### 2.3 Prototype Data Model

This generates the crystal prototype data model file associated with the fcc crystal prototype.

In [6]:
with open('A1--Cu--fcc.json', 'w') as f:
    f.write("""{
    "crystal-prototype": {
        "identifier": {
            "common": "face-centered cubic", 
            "tag": "fcc", 
            "prototype": "Cu", 
            "Pearson-symbol": "cF4", 
            "Strukturbericht": "A1"
        }, 
        "space-group": {
            "number": 225, 
            "Hermann-Maguin": "F m -3 m", 
            "Schoenflies": "O^5_h", 
            "Wykoff": {
                "letter": "a", 
                "multiplicity": 4
            }
        }, 
        "atomic-system": {
            "cell": {
                "cubic": {
                    "a": {
                        "value": 1.0, 
                        "unit": "scaled"
                    }
                }
            }, 
            "atom": [
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.0, 
                            0.0, 
                            0.0
                        ], 
                        "unit": "scaled"
                    }
                }, 
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.0, 
                            0.5, 
                            0.5
                        ], 
                        "unit": "scaled"
                    }
                }, 
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.5, 
                            0.0, 
                            0.5
                        ], 
                        "unit": "scaled"
                    }
                }, 
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.5, 
                            0.5, 
                            0.0
                        ], 
                        "unit": "scaled"
                    }
                }
            ]
        }
    }
}""")

### 2.4 Stacking Fault Model

This generates the stacking fault model for a system with the (111) plane on the xy plane of the box.

In [7]:
with open('111_stacking_fault.json', 'w') as f:
    f.write("""{
    "stacking_fault-parameters": {    
        "system-family": "A1--Cu--fcc",
        "atomman-generalized-fault-parameters": {         
            "crystallographic-axes": {
                "x-axis": [0, -1, 1],
                "y-axis": [2, -1, -1],
                "z-axis": [ 1, 1, 1]
            },
            "shift": [ 0.000, 0.000, 0.000001]
        }
    }
}""")

## 3. LAMMPS Script Templates and Generation Functions

This section contains template versions of LAMMPS simulation scripts that are used by this calculation and the functions used to fill in the templates.

1. __LAMMPS Templates__: The embedded template file(s) which LAMMPS input scripts are generated from.

2. __Script Generation Functions__: The function(s) that fill in the templates using supplied parameters.

### 3.1 LAMMPS Templates
Creates the template for the LAMMPS input script.

In [8]:
with open('min.template', 'w') as f:
    f.write("""#LAMMPS input script that performs a simple energy minimization

<atomman_system_info>

<atomman_pair_info>

fix relax<cutting_axis> all setforce <relax_params>

variable peatom equal pe/atoms

#thermo_style custom step temp pe ke etotal lx ly lz 
thermo_style custom step lx ly lz pxx pyy pzz pe v_peatom
thermo_modify format float %.13e
#min_style sd

dump dumpit all custom <maximum_evaluations> atom.* id type x y z
dump_modify dumpit format "%i %i %.13e %.13e %.13e"
minimize <energy_tolerance> <force_tolerance> <maximum_iterations> <maximum_evaluations>""")

### 3.2 Script Generation Functions
Fills in the above template with the correct values.

In [9]:
def sf_relax_script(template_file, system_info, pair_info, etol = 0.0, ftol = 1e-6, maxiter = 100000, maxeval = 100000, 
                    cutting_axis = 'z', relax_params_in = '0.0 0.0 NULL'):
    """Create lammps script for performing a simple energy minimization."""    
    
    with open(template_file) as f:
        template = f.read()
    variable = {'atomman_system_info': system_info,
                'atomman_pair_info':   pair_info,
                'energy_tolerance': etol, 
                'force_tolerance': ftol,
                'maximum_iterations': maxiter,
                'maximum_evaluations': maxeval,
                'cutting_axis': cutting_axis,
                'relax_params': relax_params_in
               }
    return '\n'.join(iprPy.tools.fill_template(template, variable, '<', '>'))


## 4. Python Calculation Function(s)

In [10]:
def sf_run_calcs(input_dict, __calc_type__, step):  
    """This calculation takes atomic positions for an already-rotated and supersized system and applies the necessary shift to 
    calculate the stacking fault energy. The code first sets up the system depending on the step, performs the calculation, and
    returns both the area and the cohesive energy of the system for that step."""
    
    #--------------------------CHECKS------------------------
    #CHECK: make sure that the cutting axis definition matches up with the shift values
    if input_dict['cutting_axis'] == 'z':
        assert int(input_dict['shift_amount'].split()[2]) == 0, "Ensure that the z-coordinate in <shift_amount> = 0!"
    elif input_dict['cutting_axis'] == 'y':
        assert int(input_dict['shift_amount'].split()[1]) == 0, "Ensure that the y-coordinate in <shift_amount> = 0!"
    elif input_dict['cutting_axis'] == 'x':
        assert int(input_dict['shift_amount'].split()[0]) == 0, "Ensure that the x-coordinate in <shift_amount> = 0!"
    
    #CHECK: cutting_plane_location value is a fraction of the entire supercell
    assert 0 < float(input_dict['cutting_plane_location']) < 1, 'cutting_plane_location must be a value between 0 and 1!'
        
    #-------------------SETS UP THE SYSTEM--------------------
    #searches within stacking_fault_model for the string 'atomman-generalized-fault-parameters'
    sf_params = input_dict['stacking_fault_model'].find('atomman-generalized-fault-parameters')
    
    #Read in potential
    potential = lmp.Potential(input_dict['potential'], input_dict['potential_dir']) #reads the potential filename
    system = input_dict['initial_system']

    #change boundary condition and relaxation parameters depending on the step
    if step == 'change_bc' or step == 'shift':
        if input_dict['cutting_axis'] == 'z':
            x_bool = True #periodic
            y_bool = True #periodic
            z_bool = False #free surface
            axis_1 = 'x-axis'
            axis_2 = 'y-axis'
            relax_params = '0.0 0.0 NULL'

        elif input_dict['cutting_axis'] == 'x':
            x_bool = False
            y_bool = True
            z_bool = True
            axis_1 = 'y-axis'
            axis_2 = 'z-axis'
            relax_params = 'NULL 0.0 0.0'

        elif input_dict['cutting_axis'] == 'y':
            x_bool = True
            y_bool = False
            z_bool = True
            axis_1 = 'x-axis'
            axis_2 = 'z-axis'
            relax_params = '0.0 NULL 0.0'

        #set boundary conditions for both change_bc and shift steps
        system.pbc = [x_bool, y_bool, z_bool]

        #perform shifting if on the shift step
        if step == 'shift':
                                   
            #calculate periodicity along the 2 axis directions because one repeat unit can be periodic at the halfway point
            shift_divisor_1 = np.sum(np.absolute(sf_params['crystallographic-axes'][axis_1][:])) 
            shift_divisor_2 = np.sum(np.absolute(sf_params['crystallographic-axes'][axis_2][:]))
                        
            system_dim = np.array([system.box.avect[0]/abs(input_dict['size_mults'][0][0]-input_dict['size_mults'][0][1]),
                                  system.box.bvect[1]/abs(input_dict['size_mults'][1][0]-input_dict['size_mults'][1][1]),
                                  system.box.cvect[2]/abs(input_dict['size_mults'][2][0]-input_dict['size_mults'][2][1])])
        
            #store initial position
            pos = system.atoms_prop(key='pos')

            #define cutting plane at a user-specified fraction of the entire system and then shift initial positions as necessary
            if input_dict['cutting_axis'] == 'z':                
                cutting_plane = ((system.box.zhi + system.box.zlo)*float(input_dict['cutting_plane_location']))
                pos[pos[:,2]>cutting_plane - 1e-9] += np.array([float(input_dict['shift_amount'].split()[0])*system_dim[0]/shift_divisor_1, 
                                                    float(input_dict['shift_amount'].split()[1])* system_dim[1]/shift_divisor_2, 0.0])
            elif input_dict['cutting_axis'] == 'y':
                cutting_plane = ((system.box.yhi + system.box.ylo)*float(input_dict['cutting_plane_location']))
                pos[pos[:,1]>cutting_plane - 1e-9] += np.array([float(input_dict['shift_amount'].split()[0])*system_dim[0]/shift_divisor_1, 0.0, 
                                                    float(input_dict['shift_amount'].split()[2])* system_dim[2]/shift_divisor_2,])
            elif input_dict['cutting_axis'] == 'x':
                cutting_plane = ((system.box.xhi + system.box.xlo)*float(input_dict['cutting_plane_location']))
                pos[pos[:,0]>cutting_plane - 1e-9] += np.array([0.0, float(input_dict['shift_amount'].split()[1])*system_dim[1]/shift_divisor_1, 
                                                    float(input_dict['shift_amount'].split()[2])* system_dim[2]/shift_divisor_2])
            #assign shifted values to the system
            system.atoms_prop(key='pos', value=pos)

    elif step == 'initial':
        system.pbc = [True, True, True]
        relax_params = 'NULL NULL NULL'
    
    #ensure all atoms are within the box
    system.wrap()
    
    #-------------------RUN LAMMPS AND SAVE RESULTS------------------
    #use above information to generate system_info and pair_info for LAMMPS
    system_info = am.lammps.atom_data.dump('stacking_fault.dat', system, units=potential.units, atom_style=potential.atom_style)
    pair_info = potential.pair_info(input_dict['symbols']) #pulls the potential info when given which element the calculation is running on

    #write the LAMMPS input script
    with open('stacking_fault.in', 'w') as f:
        f.write(sf_relax_script('min.template', system_info, pair_info, 
                                etol = input_dict['energy_tolerance'], 
                                ftol = input_dict['force_tolerance'], 
                                maxiter = input_dict['maximum_iterations'], 
                                maxeval = input_dict['maximum_evaluations'],
                                cutting_axis = input_dict['cutting_axis'],
                                relax_params_in = relax_params))
    
    #run LAMMPS
    output = lmp.run(input_dict['lammps_command'], 'stacking_fault.in', input_dict['mpi_command'])
    atom_last = 'atom.%i' % output.finds('Step')[-1] #prints number of iterations (?)
    
    #save results into results_dict
    if step == 'initial':
        try:
            os.rename(atom_last, 'initial.dump')
        except:
            os.remove('initial.dump')
            os.rename(atom_last, 'initial.dump')
        os.remove('atom.0')
        d_system = lmp.atom_dump.load('initial.dump')

    elif step == 'change_bc':
        try:
            os.rename(atom_last, 'surface.dump')
        except:
            os.remove('surface.dump')
            os.rename(atom_last, 'surface.dump')
        os.remove('atom.0')
        d_system = lmp.atom_dump.load('surface.dump')

    elif step == 'shift':
        try:
            os.rename(atom_last, 'gpfe.dump')
        except:
            os.remove('gpfe.dump')
            os.rename(atom_last, 'gpfe.dump')
        os.remove('atom.0')
        d_system = lmp.atom_dump.load('gpfe.dump')

    results_dict = {}
    results_dict['defect_system'] = d_system
    results_dict['potential_energy'] = float(output.finds('PotEng')[-1])
    
    #calculate area for output
    if input_dict['cutting_axis'] == 'z':
        area = (system.box.xhi-system.box.xlo)*(system.box.yhi-system.box.ylo)

    elif input_dict['cutting_axis'] == 'y':
        area = (system.box.xhi-system.box.xlo)*(system.box.zhi-system.box.zlo) 
    elif input_dict['cutting_axis'] == 'x':
        area = (system.box.yhi-system.box.ylo)*(system.box.zhi-system.box.zlo)

    #Save data model of results 
    results = iprPy.calculation_data_model(__calc_type__, input_dict, results_dict)

    #return values
    return [results_dict['potential_energy'], area, system.natoms]


In [11]:
def sf_return_results(input_dict,__calc_type__):
    #initialize
    results_dict = {}
    
    #run for each step and store results
    initial_results = sf_run_calcs(input_dict, __calc_type__, 'initial') 
    change_bc_results = sf_run_calcs(input_dict, __calc_type__, 'change_bc')
    shift_results = sf_run_calcs(input_dict, __calc_type__, 'shift')
    
    #process results to return in the units of interest    
    results_dict['cohesive_energy'] = initial_results[0]/initial_results[2] #eV/atom
    results_dict['surface_energy'] = abs(initial_results[0]-change_bc_results[0])/(2*change_bc_results[1]) #eV/angstrom^2
    results_dict['fault_energy'] = abs(shift_results[0]-change_bc_results[0])/(2*shift_results[1]) #eV/angstrom^2
    
    return results_dict

## 5. Run Calculation and Display Results

### 5.1 Run Calculation

In [12]:
#Read in parameters from input file
with open('calc_stacking_fault_static.in') as f:
    input_dict = iprPy.calculation_read_input('stacking_fault_static', f)
    
__calc_type__ = 'stacking_fault_static' #delete later, just a temporary assignment for the variable to successfully output a results file


### 5.2 Display Results

In [13]:
results_dict = sf_return_results(input_dict,__calc_type__)

print "Cohesive energy %f eV/atom" %results_dict['cohesive_energy']
print "Free surface energy %f eV/angstrom^2" %results_dict['surface_energy']
print "Stacking fault energy: %f eV/angstrom^2" %results_dict['fault_energy']

Cohesive energy -3.353000 eV/atom
Free surface energy 0.046933 eV/angstrom^2
Stacking fault energy: 0.017944 eV/angstrom^2
